In [300]:
import torch
import torch.utils.data.dataloader
import torch.utils.data.dataset
import torch.nn as nn

In [301]:
class Config_class:
    
    def __init__(self,config):
        self.config = config


class LM(nn.Module):
    
    def __init__(self,config):
        super().__init__()
        self.config = config
        # This will tak token ids an make enbedding out of it
        self.embedding = nn.Embedding(num_embeddings=self.config['vocab_size'],embedding_dim=self.config['embedding'],padding_idx=989)
        self.LSTM = nn.LSTM(num_layers=self.config['num_layers'],input_size=self.config['embedding'],hidden_size=self.config['hidden'])
        self.convert_vocab = nn.Linear(in_features=self.config['hidden'],out_features=self.config['vocab_size'])
        self.softmax = nn.Softmax(dim=1)
    def forward(self,token_seq):
        x = self.embedding(token_seq)
        out,_ = self.LSTM(x) # (h_n,c_n)
        convert_output = self.convert_vocab(out)
        prob = self.softmax(convert_output)
        return prob
    

### Loading dataset and train test split

In [302]:
pp_text = open('../data/clean_pp.txt').readlines() 

In [303]:
import random
from random import shuffle
random.seed(42)
shuffle(pp_text)
trl = int(0.7*l)
vrl = int(0.15*l)
train = pp_text[:trl]
val = pp_text[trl:trl+vrl]
test = pp_text[trl+vrl:]

### Making vocabulary Reserve for [PAD] , [UNK] , [STR] , [END]

In [304]:
vocab = set()
for sent in train:
    for word in sent.split():
        vocab.add(word)

In [305]:
len(vocab)

5925

In [306]:
res = 4
word_idx = {k:v+res for (v,k) in enumerate(vocab)}
word_idx['[PAD]'] = 0
word_idx['[STR]'] = 1
word_idx['[END]'] = 2
word_idx['[UNK]'] = 3

In [307]:
vocab = set(word_idx.keys())

In [308]:
len(vocab)

5929

In [309]:
idx_word =  { v:k for k,v in word_idx.items() }

In [310]:
idx_word[3]

'[UNK]'

In [311]:
def tokenizer(text):
    tokens = []
    text = text.strip('\n')
    text = idx_word[1]+' '+text+' '+idx_word[2]
    # print(text)
    for word in text.split():
        if word in word_idx.keys():
            tokens.append(word_idx[word])
        else:
            tokens.append(word_idx['[UNK]']) # update []
    return tokens

In [312]:
from torch.utils.data import Dataset
class Dataset(Dataset):
    def __init__(self,data,vocab_size):
        self.data = data
        self.vocab_size = vocab_size
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        out = tokenizer(self.data[idx])
        inputs = out[:-1]# Last token is [END] will not include in data
        labels = out[1:]# First token is [STR] will never be predected
        return torch.tensor(inputs),torch.nn.functional.one_hot(torch.tensor(labels),num_classes=self.vocab_size)

In [313]:
train_dataset = Dataset(train,len(vocab))
val_dataset = Dataset(val,len(vocab))
test_dataset = Dataset(test,len(vocab))

In [314]:
a,b = train_dataset[0]

In [315]:
a.shape

torch.Size([61])

In [316]:
b.shape

torch.Size([61, 5929])

In [317]:
len(train_dataset)

1553

In [318]:
len(val)

332

In [319]:
len(test_dataset)

334

In [320]:
from torch.utils.data import DataLoader
train_dl =  DataLoader(train_dataset,shuffle=True)
val_dl = DataLoader(val_dataset)
test_dl = DataLoader(test_dataset)

In [321]:
LM_config = Config_class({
    'vocab_size': len(vocab),
    'embedding':128,
    'hidden':256,
    'lr':0.0001,
    'num_layers':1,
    'pad_idx':word_idx['[PAD]']
})

In [322]:
LM_config.config

{'vocab_size': 5929,
 'embedding': 128,
 'hidden': 256,
 'lr': 0.0001,
 'num_layers': 1,
 'pad_idx': 0}

In [323]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        print(pred.shape)
        print(y.shape)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [324]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [325]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
LM_1 = LM(config=LM_config.config)
LM_1.to(device)
loss = nn.CrossEntropyLoss()
optmizer = torch.optim.Adam(params = LM_1.parameters())

In [326]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dl, LM_1, loss, optmizer)
    test(test_dl, LM_1, loss)
print("Done!")

Epoch 1
-------------------------------
torch.Size([1, 50, 5929])
torch.Size([1, 50, 5929])


RuntimeError: Expected floating point type for target with class probabilities, got Long